In [3]:
#!/bin/python3

#!/bin/python3

import os
import sys
import re

hash = {}
class Current:
    def __init__(self,order_id,timestamp,symbol,order_type,side,price,quantity):
        self.order_id = order_id
        self.timestamp = timestamp
        self.symbol = symbol
        self.order_type = order_type
        self.side = side
        self.price = price
        self.quantity = quantity
        self.ismatched = 0
        
class OrderList:
    orderBook = {} #dictionary for all orders where key is order id
    time = 0
    
# Complete the function below.

def Type(string):
    patInt = re.search('\d+',string)
    patFloat = re.search('\d+\.\d+', string)
    patStr = re.search('\S+', string)
    if(patFloat):
        return float
    elif(patInt):
        return int
    elif(patStr):
        return str

def splitN(q):
    #  A,2,0000001,XYZ,L,B,103.53,150
    return [int(q.split(',')[1]),int(q.split(',')[2]),(q.split(',')[3]),(q.split(',')[4]),(q.split(',')[5]),float(q.split(',')[6]),int(q.split(',')[7])]


def ammend_command(command):
    delete_all_matched_orders()
    params = command.split(",")
    flag = True
    if len(params) != 8:
        return RequestStatus.ORDER_REJECT
    for i in [1,2,7]:
        try:
            params[i] = int(params[i])
        except:
            print("can not convert to int",str(params[i]))
           
    params[6] = float(params[6])
            
    _id = params[1]
    time = params[2]
    symbol = params[3]
    _type = params[4]
    side = params[5]
    price = params[6]
    quantity = params[7]
    
    if(type(_id)!=int or type(time)!=int or type(quantity)!=int):
        print("data type incorrect")
        flag = False
    
    if(type(price)!=float and type(price)!=int):
        print("price bad format")
        flag = False
    
    if(price<0):
        flag = False
    
    if(_type != "M" and price<=0 ):
        flag = False
    
    if(_type not in ORDER_TYPES):
        print("incorrect order type")
        flag = False
    
    if(side not in SIDE_TYPES):
        print("incorrect side type")
        flag = False
    
    if(_id not in AllOrders.orderBook):
        return format_id(_id)+RequestStatus.AMEND_REJECT_ORDER_DOES_NOT_EXIST
    
    order = AllOrders.orderBook[_id]
    if(order.symbol!= symbol or order.side != side or order._id!=_id or order._type!=_type):
        print("Can not ammend these details")
        flag = False
    
    if(time<AllOrders.latestTime):
        flag = False
    
    if flag:
        if order.matched >= quantity:
            order.quantity = 0
        else:
            order.quantity = quantity - order.matched
        order.price = price
        #order.timestamp = time
        AllOrders.latestTime = time
        AllOrders.orderBook[_id] = order #changing the object in dictionary
        return format_id(_id)+RequestStatus.AMEND_ACCEPT
            
    else:
        return format_id(_id)+RequestStatus.AMEND_REJECT_INVALID_DETAILS
    
    

def check_type(query):
    l = query.split(',')
    if(Type(l[1]) is int and
      Type(l[2]) is int and
      Type(l[3]) is str and l[3].isupper() and
      (l[4]=='L' or l[4]=='M' or l[4]=='I') and
      (l[5]=='S' or l[5]== 'B') and
      Type(l[6]) is float and
      (float(l[6])>0 and int(l[7])>0) and
      Type(l[7]) is int):
        return True
    return False

def cancel_order(query):
    orderId = int(query.split(",")[1])
    if(orderId in list(OrdersList.orderBook)):
        del OrdersList.orderBook[orderId]
        return str(orderId) " - " +"CancelAccept" 
    return str(orderId)+ " - " +"CancelReject - 404 - Order does not exist"

def new_order(q):
    s = ''
    orderId = int(q.split(',')[1])
    time = int(q.split(',')[2])
    if(check_type(q) and not orderId in OrderList.orderBook and len(q.split(','))==8 and time >= OrderList.time):
        s = str(orderId) + ' - ' + "Accept"
        hash[orderId] = splitN(q)
        order = splitN(q)
        OrderList.orderBook[orderId] = Current(order[0],order[1],order[2],order[3],order[4],order[5],order[6])
        OrderList.time = time
    else:
        s = str(orderId) + ' - ' + "Reject - 303 - Invalid order details"
    return s

    
def processQueries(queries):
    lst = []
    for q in queries:
        check = q.split(',')[0]
        if(check=='N'):
            lst.append(new_order(q))

        if(check=='X'):
            lst.append(cancel_order(q))

        if(check=='A'):
            lst.append(amend_order(q))

        if(check=='M'):
            lst.append(match_order(q))

        if(check=='Q'):
            lst.append(query_order(q)) 
    return lst
            

if __name__ == '__main__':
    f = open(os.environ['OUTPUT_PATH'], 'w')

    queries_size = int(input())

    queries = []
    for _ in range(queries_size):
        queries_item = input()
        queries.append(queries_item)

    response = processQueries(queries)
    f.write("\n".join(response))

    f.write('\n')

    f.close()